## Plot mt variants on nuclear embedding

In [1]:
# Input info
se_f = "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_A2_black/annotation_clones/SE.rds"
cells_meta_f = "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/mt_clones/thr__0.01_rt__10/cells_mt.tsv"
outdir = "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_A2_black/annotation_clones/de_clone_btwnvars_RNA_af/"

ids.reversed = T #reverse IDs from suffix to prefix

# Parallel processing
cores = 4
# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"


In [2]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")
library(dplyr)
library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Attaching SeuratObject

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required pa

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [3]:
source(utils_script)

In [4]:
dir.create(outdir)

Warning message in dir.create(outdir):
"'/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_A2_black/annotation_clones/de_clone_btwnvars_RNA_af' already exists"


In [5]:
cells_meta <- read.table(cells_meta_f, sep="\t", header = T, row.names="cell")
cells_meta 

#cells_meta
ids.reversed = T

if (ids.reversed){
    print('here')
    split.names <- stringr::str_split(rownames(cells_meta), "_")
    rownames(cells_meta) = lapply(split.names, function(x){paste0(x[2], "_", x[1])})
}
cells_meta = cells_meta[, order(colSums(-cells_meta))]
head(cells_meta)

,X10463C,X10559G,X11251G,X11453A,X11719A,X11812G,X13368A,X14233G,X14674C,X14905A,⋯,X196C,X204C,X2442C,X3847C,X58C,X64T,X7598A,X827G,X8292A,X8461T
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAACGAAAGAATCAAC-1_Control,0.00000000,0.00000000,0.15384615,0.00000000,0.00000000,0.05405405,0,0.0000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAAAGACGCCAA-1_Input,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAAAGCGAGAAA-1_Control,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAAAGTACAGAT-1_Control,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAAAGTTAGCAA-1_Control,0.00000000,0.00000000,0.00000000,0.00000000,0.42857143,0.00000000,0,0.0000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAACAAGCGAAC-1_Flt3l,0.00000000,0.00000000,0.00000000,0.00000000,0.11111111,0.00000000,0,0.0000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAACACAGCCAC-1_Flt3l,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAACACAGGTAG-1_Flt3l,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGAACACGCTCAG-1_Input,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0,0.00000000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


[1] "here"


,X9899C,X5581G,X1949A,X11453A,X3109C,X15297C,X2623G,X3244A,X16129A,X152C,⋯,X196C,X204C,X2442C,X3847C,X58C,X64T,X7598A,X827G,X8292A,X8461T
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Control_AAACGAAAGAATCAAC-1,0.00000000,1,0.00000000,0,0,0,0.0000000,0,0,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Input_AAACGAAAGACGCCAA-1,0.00000000,0,0.02877698,0,0,0,0.0000000,0,0,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Control_AAACGAAAGCGAGAAA-1,0.07692308,0,0.00000000,0,0,0,0.3333333,0,0,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Control_AAACGAAAGTACAGAT-1,0.00000000,0,0.00000000,0,0,0,0.0000000,0,0,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Control_AAACGAAAGTTAGCAA-1,0.00000000,0,0.11764706,0,0,0,0.0000000,0,0,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Flt3l_AAACGAACAAGCGAAC-1,0.00000000,0,0.00000000,0,0,0,0.0000000,0,0,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [6]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
se

An object of class Seurat 
221404 features across 18672 samples within 2 assays 
Active assay: RNA (19039 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

### Subset to just in cells_meta

In [7]:
se$inClones = 0
se[["inClones"]][(colnames(se) %in% row.names(cells_meta)), "inClones"] = 1

sum(se[["inClones"]])/dim(se[[]][1])

se.filt <- subset(se, subset = inClones == 1 )

[1] 8.902635e-01 1.662300e+04

In [8]:
se.filt <- AddMetaData(se.filt, cells_meta)

In [9]:
se

An object of class Seurat 
221404 features across 18672 samples within 2 assays 
Active assay: RNA (19039 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [10]:
se.filt

An object of class Seurat 
221404 features across 16623 samples within 2 assays 
Active assay: RNA (19039 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [11]:
source(utils_script)
tmp <- get.clone.sizes(se.filt)
counts.conds.norm <- tmp[["counts.norm"]]
clone.sizes <- tmp[["clone.size"]]
head(clone.sizes)

`summarise()` has grouped output by 'condition', 'donor', 'lineage'. You can override using the `.groups` argument.

Warning message in xtfrm.data.frame(x):
"cannot xtfrm data frames"
`summarise()` has grouped output by 'donor'. You can override using the `.groups` argument.



donor,lineage,norm.total,total,name,cdf.norm,cdf,index
<fct>,<fct>,<dbl>,<int>,<fct>,<dbl>,<int>,<int>
0,2,0.1389416,384,0_2,0.06947081,384,1
0,0,0.1355984,404,0_0,0.13726999,788,2
0,1,0.1349289,410,0_1,0.20473445,1198,3
0,3,0.1170903,339,0_3,0.26327961,1537,4
0,4,0.1045133,293,0_4,0.31553624,1830,5
0,6,0.1039419,304,0_6,0.36750721,2134,6


## plot each clone pre and post 

In [12]:
mt.vars <- colnames(cells_meta)
mt.vars

[1] "X9899C"  "X5581G"  "X1949A"  "X11453A" "X3109C"  "X15297C" "X2623G" 
 [8] "X3244A"  "X16129A" "X152C"   "X5442C"  "X8743A"  "X15497A" "X14674C"
[15] "X10463C" "X10559G" "X11251G" "X11719A" "X11812G" "X13368A" "X14233G"
[22] "X14905A" "X150T"   "X15452A" "X15607G" "X15928A" "X16294T" "X16296T"
[29] "X16304C" "X16519C" "X1888A"  "X4117C"  "X4216C"  "X4917G"  "X5147A" 
[36] "X5580C"  "X709A"   "X8697A"  "X930A"   "X10397G" "X10589A" "X11761T"
[43] "X13188T" "X146C"   "X16355T" "X16362C" "X196C"   "X204C"   "X2442C" 
[50] "X3847C"  "X58C"    "X64T"    "X7598A"  "X827G"   "X8292A"  "X8461T"

In [13]:
for (mt in (mt.vars)){
    print(mt)    
    p <- FeaturePlot(se.filt,features=mt, split.by = "condition", order=T) & theme(legend.position = c(0.1,0.2))
    curr.count = sum(se.filt[[mt]]>0, na.rm=TRUE)
    ggsave(file.path(outdir, paste("withNA.count", curr.count,mt, "groupCond.png", sep="_")), p)
    
    # Run with removing NA 
    expr <- FetchData(se.filt, vars = mt)
    curr.se <- se.filt[, which((!is.na(expr)))]
    p <- FeaturePlot(curr.se,features=mt, split.by = "condition", order=T) & theme(legend.position = c(0.1,0.2))
    curr.count = sum(curr.se[[mt]]>0, na.rm=TRUE)
    ggsave(file.path(outdir, paste("count", curr.count,mt, "groupCond.png", sep="_")), p)
    
}

[1] "X9899C"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X5581G"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X1949A"


...

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X8292A"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "X8461T"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [14]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.3 LTS

Matrix products: default
BLAS/LAPACK: /home/isaac/miniconda3/envs/mttrace/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] future_1.23.0             dplyr_1.0.7              
 [3] RColorBrewer_1.1-2        metap_1.1                
 [5] cowplot_1.1.1             magrittr_2.0.1           
 [7] data.table_1.14.2         patchwork_1.1.1          
 [9] ggplot2_3.3.5             EnsDb.Hsapiens.v75_2.99.0
[11] 

In [15]:
outdir

[1] "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_A2_black/annotation_clones/de_clone_btwnvars_RNA_af/"